In [168]:
import ipynb.fs.defs.functions as fct
import pandas as pd
import numpy as np
import pickle
import random
from scipy.stats import pearsonr
from scipy.stats import bernoulli

# Sim - refit

In [169]:
all_users_folder = 'data/all_users/'

# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

In [170]:
# General
model = 'mod12'

# Number of simulations
Nsim = 100

In [171]:
# Model settings
mod_info = pd.read_pickle(all_users_folder + model + '/mod_parameters.pkl')
print(mod_info)

# Extract functions
value_fct = getattr(fct, mod_info['value_fct'])
dec_fct = getattr(fct, mod_info['dec_fct'])

# Parameter bounds
param_lower_bounds = [-2, 0, 0, 0]
param_upper_bounds = [ 2, 1, 2.5, 3]

{'name': 'model12', 'value_fct': 'rescorla_wagner_reinitV0', 'dec_fct': 'my_softmax_shrinking_press_bias', 'param_names': ['v0r', 'alpha', 'beta', 'pi_t']}


In [172]:
def rescorla_wagner_reinitV0_1trial(vt_m_1, isHit, fb, Ntrials, t, alpha, v0r):    

    pe = np.nan
    
    # Start of run 2, reinitialise v0
    if t == int(Ntrials/2): 
        vt = v0r

    # if hit, recieves fb
    elif isHit == 1:
        # Compute prediction error
        pe = fb - vt_m_1
        # Compute new vt and fill in 
        vt = vt_m_1 + alpha * pe

    # if no hit, no fb
    elif isHit == 0:
        # vt does not change 
        vt = vt_m_1

    return vt, pe

In [173]:
def compute_shrink_factors(trialNo_all_cues, cue):
    
    # Extract trials for this cue
    trials = trialNo_all_cues[cue]

    # Reinitialise each trial of run 2 (above 56)
    trials_per_run_lst = []
    run_size = 56
    for trial in trials:
        if trial > run_size:
            trials_per_run_lst.append(trial-run_size)
        else:
            trials_per_run_lst.append(trial)

    # Convert to array
    trials_per_run = np.array(trials_per_run_lst)

    # Compute shrinking factor = (N-t)/N
    shrink = (run_size - trials_per_run)/run_size

    return shrink

In [174]:
def my_softmax_shrinking_press_bias_1trial(shrink, pi_t, beta, vt):

    # Higher press bias with small trial numbers
    shrinking_pi = pi_t * shrink
    x = beta * (vt + shrinking_pi)

    try:
        p_hit =  np.exp(x)/(np.exp(x)+1)

    except RuntimeWarning:
        # to avoid overflow errors
        expon_bound = 700
        p_hit = [1 if el>expon_bound else (np.exp(el)/(np.exp(el)+1)) for el in x]
    
    isHit = bernoulli.rvs(p_hit, size=1)

    return int(isHit)

In [175]:
def simulate_behav(fbs_all_cues, trialNo_all_cues, param_values, param_names):
    
    # Value function 
    # Free parameters
    v0r = param_values[param_names.index('v0r')]
    alpha = param_values[param_names.index('alpha')]
    
    Ntrials = sum(len(lst) for lst in fbs_all_cues.values())

    # Initialise empty dictionary
    vt_all_cues = dict.fromkeys(fbs_all_cues.keys())
    pe_all_cues = dict.fromkeys(fbs_all_cues.keys())
    
    # Decision function
    # Free parameters
    beta = param_values[param_names.index('beta')]
    pi_t = param_values[param_names.index('pi_t')]

    # Initialise empty dictionary
    isHit_all_cues = dict.fromkeys(vt_all_cues.keys())
    
    # Iterate over cues
    for cue, fbs in zip(fbs_all_cues.keys(), fbs_all_cues.values()):

        # Initialise vector of values
        vts = np.empty(len(fbs))
        vts.fill(np.nan)

        # Initialise vector of hits
        hits = np.empty(len(fbs))
        hits.fill(np.nan)

        # Initialise vector of PEs
        pes = np.empty(len(fbs))
        pes.fill(np.nan)

        # Fill in prior
        # If want specific prior per cue, check value of cue
        vts[0] = v0r
        hits[0] = 0

        # Compute shrink factors for decision function
        shrink = compute_shrink_factors(trialNo_all_cues, cue)

        # Iterate to fill in vector
        for t in range(1,len(vts)):
            vt, pe = rescorla_wagner_reinitV0_1trial(vts[t-1], hits[t-1], fbs[t-1], Ntrials, t, alpha, v0r)
            isHit = my_softmax_shrinking_press_bias_1trial(shrink[t], pi_t, beta, vt)
            vts[t] = vt
            pes[t] = pe
            hits[t] = isHit

        vt_all_cues[cue] = vts
        pe_all_cues[cue] = pes
        isHit_all_cues[cue] = hits
        
    return vt_all_cues, pe_all_cues, isHit_all_cues

In [176]:
# Get parameter values for simulation

# Load fitted parameters
data_mod = pd.read_pickle(all_users_folder + model + '/mod_param_fits.pkl')
parameter_means = data_mod[mod_info['param_names']].mean()
parameter_stds = data_mod[mod_info['param_names']].std()

# Initialise matrices of nans
all_sim_param_values = np.empty((Nsim, len(mod_info['param_names'])))
all_fit_param_values = np.empty((Nsim, len(mod_info['param_names'])))
all_sim_param_values[:] = np.NaN
all_fit_param_values[:] = np.NaN

# Sim and refit
for sim_id in range(0, Nsim):
    
    if sim_id%25==0:
        print(sim_id)
    
    # Sample from mean of fitted values
    params = np.array([])

    for param_mean, param_std, low_b, upp_b in zip(parameter_means, parameter_stds, param_lower_bounds, param_upper_bounds):

        # Sample parameter from mean (resample if not within the bounds)
        param = np.nan
        while not(param >= low_b and param <= upp_b):
            param = np.random.normal(size = 1, loc = param_mean, scale = param_std)

        # Parameter value
        params = np.append(params, param)

    # Store
    mod_info['param_values'] = params

    # Create a model object
    mod = fct.Model(mod_name = mod_info['name'],
                 value_fct = value_fct, 
                 dec_fct = dec_fct, 
                 param_names = mod_info['param_names'])

    # Extract random data set
    random.shuffle(uniqueIDs)
    ID = uniqueIDs[0]
    user_folder = 'data/user_' + ID + '/'
    df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
    _, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

    # Simulate behaviour
    #mod.simulate_behaviour(fbs_all_cues, trialNo_all_cues, mod_info['param_values'], mod_info['param_names'])
    vt_all_cues, pe_all_cues, isHit_all_cues = simulate_behav(fbs_all_cues, trialNo_all_cues, mod_info['param_values'], mod_info['param_names'])
    
    # Input data to model
    mod.gen_param_values = mod_info['param_values']
    mod.set_data('', fbs_all_cues, isHit_all_cues, trialNo_all_cues)    

    # Fit model
    mod.fit(param_lower_bounds, param_upper_bounds, n_iterations=5)

    # Store
    all_sim_param_values[sim_id,:] = mod.gen_param_values
    all_fit_param_values[sim_id,:] = mod.param_values

# Reformat
df_sim = pd.DataFrame(all_sim_param_values, columns = mod_info['param_names'])
df_sim.insert(0, 'Type', 'Sim')
df_sim = df_sim.rename_axis('simID').reset_index()

df_fit = pd.DataFrame(all_fit_param_values, columns = mod_info['param_names'])
df_fit.insert(0, 'Type', 'Fit')
df_fit = df_fit.rename_axis('simID').reset_index()

df = pd.concat([df_sim, df_fit])
df.sort_values(by='simID', ascending=True, inplace = True)
df.reset_index(inplace = True, drop = True)

# Save
df.to_pickle(all_users_folder + '/sim_refit/' + mod_info['name'] + '.pkl')

0


/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/

25


/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/

50


/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/

75


/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: overflow encountered in exp
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/functions.ipynb:656: RuntimeWarning: invalid value encountered in true_divide
  "            if hits[t-1] == 1:\n",
/Volumes/GoogleDrive/Mon Drive/PAT/data_analysis/

In [177]:
mod.gen_param_values

array([0.71409365, 0.26097112, 0.46667833, 0.31180809])